# Example of Staging Upload to ImageCalc Table
This notebook goes through an example of uploading a classified planet image, a logfile, and training data to the DSWx calval database. Data files and metadata are uploading to a staging bucket. Then, the database manager will commit these uploads to the database

In [ ]:
import geopandas as gpd
import pandas as pd
import boto3
import os
from datetime import datetime
import sys
sys.path.insert(0, './tools/')
from addImageCalc import addImageCalc
from pathlib import Path
import rasterio
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds

In [ ]:
#Only specify one. Leave the other as ''. If more than one planet image for given chip, PLANET_ID must be specified 
PLANET_ID = '20211003_161639_91_241d'
SITE_NAME = ''
assert((len(PLANET_ID) == 0) ^ (len(SITE_NAME) == 0))


In [ ]:
#Local directory where classification file(s) are located
uploadDir = Path(f'planet_images_cropped/{PLANET_ID}').absolute()

#Name of classified geotif
classified_image_filename = f'classification_{PLANET_ID}.tif' 

#Uncomment if logfile exists and esnure it is included in filePaths below before upload
#log_filename = 'logfile.txt 

#Uncomment if uploading any additional files and esnure they are included in filePaths below before upload
#additional_filename = 'additional_file.txt' 

In [ ]:
# Classified Image read and geometry

with rasterio.open(uploadDir / classified_image_filename) as ds:
    bounds = ds.bounds
    crs_utm = ds.crs
    
bounds_4326 = transform_bounds(crs_utm, CRS.from_epsg(4326), *bounds)
classified_geometry = box(*bounds_4326)

In [ ]:
#Name of person who created and edited the classification
editor_name = 'Charlie Marshak' 

#Name of person who reviewed the classification. Leave as None if classification has not been reviewed
reviewer_name = '' 

#Usually 'Mannual classification' or 'Review'
calc_type = 'Manual classification' 

#Change to 'Final' if review is classification passed review with no changes
processing_level = 'Intermediate' 

#Processing notes. e.g. 'Supervised classification using SCP mannual edits using Serval informed by Pekel water mask'
notes = 'Fixed geometry crs (CM).'

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credenitals are needed

In [ ]:
bucket_name = 'opera-calval-database-dswx'
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### Read Image metadata table
To get the geometry metadata for the classified image, we copy the geometry of the source image from the database since the extents are the same. This geometry could also be generated directly from the classified imagery.

In [ ]:
imageTable = gpd.read_file(s3.Object(bucket_name,'image.geojson').get()['Body'])
imagecalcTable = gpd.read_file(s3.Object(bucket_name,'image_calc.geojson').get()['Body'])

In [ ]:
temp = imageTable[['image_name', 'site_name']]
df_site2image = temp.set_index('site_name')
df_image2site = temp.set_index('image_name')

In [ ]:
# This cell will show the number of planet images found for a given chip. If more than one, ensure the printed Planet
# ID matches the planet image used to generate the classification
if not PLANET_ID:
    values = PLANET_ID = df_site2image.loc[SITE_NAME].tolist()
    PLANET_ID = values[0]
    print(f'There was {len(values)} planet images for this chip')
else:
    values = df_image2site.loc[PLANET_ID].tolist()
    SITE_NAME = values[0]
    print(f'There were {len(values)} chips for this planet_image')

(SITE_NAME, PLANET_ID)

In [ ]:
search = imageTable[imageTable.image_name == PLANET_ID]
planet_image = search.iloc[[0]]
geometry = planet_image.geometry.iloc[0]

In [ ]:
# This cell assigns a version number to the classification. If this is the first classification of a given planet
# image, the assigned version should be 0. Otherwise, it will increment on the latest version found in the database
search = imagecalcTable[imagecalcTable.image_name == PLANET_ID]
if len(search) == 0:
    version = 0
    previous_name = None
    print('first entry into table for ID:'+PLANET_ID+' assigning version = 0')
else:
    try:
        version = search['version'].max() + 1
        previous_name = search[search.version==search['version'].max()].image_calc_name.iloc[0]
        print('assigning version based on maximum version in table. version = '+str(version))
    except:
        version = len(search)
        previous_name = None
        print('could not read version from table. assigned based on number of matching table entries. verson = '+str(version))


### Enter the required file locations and metadata fields
To upload the classified image, we need to specify its location on the local computer (and the location of auxilary files). We also need to fill in some metadata fields. Both file paths and metadata are specified as dictionaries

In [ ]:
uploadDir.exists()

In [ ]:
filePaths = {
    'image_calc' : str(uploadDir  / classified_image_filename),
    #'logfile' : uploadDir + log_filename, #uncomment this line if uploading logfile
    #additional_file: additional_file_name #uncomment this line if uploading additional file
}
filePaths

In [ ]:
metaData = {
    'image_name':planet_image.image_name.iloc[0], #str
    'image_calc_name':planet_image.image_name.iloc[0]+'_classification_v'+str(version), #str 
    'previous_name':previous_name, #str
    'calc_type':calc_type, #str 
    'processing_level':'Intermediate', #str
    'oversight_level':None, #str,
    'calculated_by': editor_name, #str
    'reviewed_by': None, #str
    'notes' : notes,
    'version' : version,
    'public':True, #bool
    'geometry':classified_geometry, #shapely geometry
}
metaData

### Stage the image
We use a pre-defined function to upload files and metadata to the staging area. This function takes the file paths and metadata dictionaries, as well as the AWS session object as inputs

    

In [ ]:
addImageCalc(filePaths,metaData,session)